In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
from py2neo import Graph, Node, Relationship, NodeMatcher
import warnings

# Desabilitar o aviso temporariamente
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
def lowerCols(column_names):
    column_names_lower = []
    for cn in column_names:
        column_names_lower.append(cn.lower())
    return column_names_lower

# Conexões

In [3]:
instant_client = "./instantclient_21_9"
cx_Oracle.init_oracle_client(lib_dir=instant_client)
# Conectar ao banco de dados Oracle
conn = cx_Oracle.connect(user="store", password="pass", dsn="localhost:1521/xe")

# estabelecer conexão neo4j
graph = Graph("bolt://localhost:7687 ", auth=("neo4j", "password"))
matcher = NodeMatcher(graph)

# SQL

In [4]:
# Criar um cursor
cursor = conn.cursor()

### PRODUCT CATEGORIES

In [5]:
cursor.execute('SELECT * FROM PRODUCT_CATEGORIES')
product_categories = cursor.fetchall()

product_categories_df = pd.DataFrame(product_categories)
product_categories_column_names = [column[0] for column in cursor.description]
product_categories_df.columns = lowerCols(product_categories_column_names)

### PRODUCT

In [6]:
productQuery = "select pd.*, s.quantity, s.max_stock_quantity, s.unit from (select p.*, d.discount_name, d.discount_desc, d.discount_percent, d.is_active_status, d.created_at as discount_created_at, d.modified_at as discount_modified_at from product p left join discount d on d.discount_id = p.discount_id) pd, stock s where pd.product_id=s.product_id"

In [7]:
cursor.execute(productQuery)
product = cursor.fetchall()

product_df = pd.DataFrame(product)
product_column_names = [column[0] for column in cursor.description]
product_df.columns = lowerCols(product_column_names)

### EMPLOYEES

In [8]:
cursor.execute('SELECT * FROM EMPLOYEES')
employees = cursor.fetchall()

employees_df = pd.DataFrame(employees)
employees_column_names = [column[0] for column in cursor.description]
employees_df.columns = lowerCols(employees_column_names)

### DEPARTMENT

In [9]:
cursor.execute('SELECT * FROM DEPARTMENTS')
departments = cursor.fetchall()

departments_df = pd.DataFrame(departments)
departments_column_names = [column[0] for column in cursor.description]
departments_df.columns = lowerCols(departments_column_names)

### EMPLOYEES_ARCHIVE

In [10]:
cursor.execute('SELECT * FROM EMPLOYEES_ARCHIVE')
employees_archive = cursor.fetchall()

employees_archive_df = pd.DataFrame(employees_archive)
employees_archive_column_names = [column[0] for column in cursor.description]
employees_archive_df.columns = lowerCols(employees_archive_column_names)

### STORE_USERS

In [11]:
cursor.execute('SELECT * FROM STORE_USERS')
store_users = cursor.fetchall()

store_users_df = pd.DataFrame(store_users)
store_users_column_names = [column[0] for column in cursor.description]
store_users_df.columns = lowerCols(store_users_column_names)

### SHOPPING_SESSION

In [12]:
cursor.execute('SELECT * FROM SHOPPING_SESSION')
shopping_session = cursor.fetchall()

shopping_session_df = pd.DataFrame(shopping_session)
shopping_session_column_names = [column[0] for column in cursor.description]
shopping_session_df.columns = lowerCols(shopping_session_column_names)

### CART_ITEM

In [13]:
cursor.execute('SELECT * FROM CART_ITEM')
cart_item = cursor.fetchall()

cart_item_df = pd.DataFrame(cart_item)
cart_item_column_names = [column[0] for column in cursor.description]
cart_item_df.columns = lowerCols(cart_item_column_names)

### ORDER_DETAILS

In [14]:
orderDetailsQuery = "select od.*, pd.amount, pd.provider, pd.payment_status, pd.created_at as payment_created_at, pd.modified_at as payment_modified_at from order_details od, payment_details pd where od.payment_id = pd.payment_details_id"

In [15]:
cursor.execute(orderDetailsQuery)
order_details = cursor.fetchall()

order_details_df = pd.DataFrame(order_details)
order_details_column_names = [column[0] for column in cursor.description]
order_details_df.columns = lowerCols(order_details_column_names)

### ADDRESSES

In [16]:
cursor.execute('SELECT * FROM ADDRESSES')
addresses = cursor.fetchall()

addresses_df = pd.DataFrame(addresses)
addresses_column_names = [column[0] for column in cursor.description]
addresses_df.columns = lowerCols(addresses_column_names)

### ORDER_ITEMS

In [17]:
cursor.execute('SELECT * FROM ORDER_ITEMS')
order_items = cursor.fetchall()

order_items_df = pd.DataFrame(order_items)
order_items_column_names = [column[0] for column in cursor.description]
order_items_df.columns = lowerCols(order_items_column_names)

# Inserir NEO4J

### PRODUCT_CATEGORIES

In [18]:
product_categories_dict = product_categories_df.to_dict(orient='records')

product_categories_dict_rel = {}
for pc in product_categories_dict:
    product_categories_node = Node("PRODUCT_CATEGORIES",  category_id=pc['category_id'], category_name=pc['category_name'])
    graph.create(product_categories_node)
    product_categories_dict_rel[pc['category_id']] = product_categories_node

### PRODUCT

In [20]:
product_df.replace({pd.NaT: None}, inplace=True)

product_dict = product_df.to_dict(orient='records')
product_dict_rel = {}
for p in product_dict:
    product_node = Node("PRODUCT", product_id=p["product_id"], product_name=p["product_name"], category_id=p["category_id"], sku=p["sku"], price=p["price"], discount_id=p["discount_id"], created_at=p["created_at"], last_modified=p["last_modified"], discount_name=p["discount_name"], discount_desc=p["discount_desc"], discount_percent=p["discount_percent"], is_active_status=p["is_active_status"], discount_created_at=p["discount_created_at"], discount_modified_at=p["discount_modified_at"], quantity=p["quantity"], max_stock_quantity=p["max_stock_quantity"], unit=p["unit"])
    graph.create(product_node)
    
    pc_p_rel = Relationship(product_node, "category", product_categories_dict_rel[p["category_id"]])
    graph.create(pc_p_rel)
    product_dict_rel[p["product_id"]] = product_node

### DEPARTMENT

In [21]:
departments_dict = departments_df.to_dict(orient='records')
departments_dict_rel = {}

for dp in departments_dict:
    department_node = Node("DEPARTMENT", department_id=dp["department_id"], department_name=dp["department_name"])
    graph.create(department_node)
    departments_dict_rel[dp["department_id"]] = department_node

### EMPLOYEES

In [22]:
employees_dict = employees_df.to_dict(orient='records')

for e in employees_dict:
    employees_node = Node("EMPLOYEES", employee_id=e["employee_id"], first_name=e["first_name"], middle_name=e["middle_name"], last_name=e["last_name"], date_of_birth=e["date_of_birth"], department_id=e["department_id"], hire_date=e["hire_date"], salary=e["salary"], phone_number=e["phone_number"], email=e["email"], ssn_number=e["ssn_number"], manager_id=e["manager_id"])
    graph.create(employees_node)

    e_dp_rel = Relationship(employees_node, "work", departments_dict_rel[e["department_id"]])
    graph.create(e_dp_rel)

### EMPLOYEES_ARCHIVE

In [23]:
employees_archive_dict = employees_archive_df.to_dict(orient='records')

for ea in employees_archive_dict:
    employee_archive_node = Node("EMPLOYEE_ARCHIVE", event_date=ea["event_date"], event_type=ea["event_type"], user_name=ea["user_name"], old_employee_id=ea["old_employee_id"], old_first_name=ea["old_first_name"], old_middle_name=ea["old_middle_name"], old_last_name=ea["old_last_name"], old_date_of_birth=ea["old_date_of_birth"], old_department_id=ea["old_department_id"], old_hire_date=ea["old_hire_date"], old_salary=ea["old_salary"], old_phone_number=ea["old_phone_number"], old_email=ea["old_email"], old_ssn_number=ea["old_ssn_number"], old_manager_id=ea["old_manager_id"], new_employee_id=ea["new_employee_id"], new_first_name=ea["new_first_name"], new_middle_name=ea["new_middle_name"], new_last_name=ea["new_last_name"], new_date_of_birth=ea["new_date_of_birth"], new_department_id=ea["new_department_id"], new_hire_date=ea["new_hire_date"], new_salary=ea["new_salary"], new_phone_number=ea["new_phone_number"], new_email=ea["new_email"], new_ssn_number=ea["new_ssn_number"], new_manager_id=ea["new_manager_id"])
    
    graph.create(employee_archive_node)

### STORE USERS

In [24]:
store_users_dict = store_users_df.to_dict(orient='records')
store_users_dict_rel = {}

for su in store_users_dict:
    store_users_node = Node("STORE_USERS", user_id=su["user_id"], first_name=su["first_name"], middle_name=su["middle_name"], last_name=su["last_name"], phone_number=su["phone_number"], email=su["email"], username=su["username"], user_password=su["user_password"], registered_at=su["registered_at"])

    graph.create(store_users_node)
    store_users_dict_rel[su["user_id"]] = store_users_node


### ADDRESSES

In [25]:
addresses_dict = addresses_df.to_dict(orient='records')
addresses_dict_rel = {}

for ad in addresses_dict:
    addresses_node = Node("ADDRESSES", adress_id=ad["adress_id"], line_1=ad["line_1"], line_2=ad["line_2"], city=ad["city"], zip_code=ad["zip_code"], province=ad["province"], country=ad["country"])
    graph.create(addresses_node)

    addresses_dict_rel[ad["adress_id"]] = addresses_node

### ORDER DETAILS

In [26]:
order_details_df.replace({pd.NaT: None}, inplace=True)

order_details_dict = order_details_df.to_dict(orient='records')
order_details_dict_rel = {}

for od in order_details_dict:
    order_details_node = Node("ORDER_DETAILS", order_details_id=od["order_details_id"], user_id=od["user_id"], total=od["total"], payment_id=od["payment_id"], shipping_method=od["shipping_method"], delivery_adress_id=od["delivery_adress_id"], created_at=od["created_at"], modified_at=od["modified_at"], amount=od["amount"], provider=od["provider"], payment_status=od["payment_status"], payment_created_at=od["payment_created_at"], payment_modified_at=od["payment_modified_at"])
    graph.create(order_details_node)

    # RELATION ADDRESSES ORDER_DETAILS
    addr_order_rel = Relationship(order_details_node, "delivery", addresses_dict_rel[od["delivery_adress_id"]])
    graph.create(addr_order_rel)

    user_order_rel = Relationship(order_details_node, "buy", store_users_dict_rel[od["user_id"]])
    graph.create(user_order_rel)

    order_details_dict_rel[od["order_details_id"]] = order_details_node

### SHOPPING SESSION

In [27]:
shopping_session_dict = shopping_session_df.to_dict(orient='records')
shopping_session_dict_rel = {}

for ss in shopping_session_dict:
    shopping_session_node = Node("SHOPPING_SESSION", session_id=ss["session_id"], user_id=ss["user_id"], created_at=ss["created_at"], modified_at=ss["modified_at"])
    graph.create(shopping_session_node)

    user_session_rel = Relationship(shopping_session_node, "session", store_users_dict_rel[ss["user_id"]])
    graph.create(user_session_rel)

    order_details_dict_rel[ss["session_id"]] = shopping_session_node

### CART_ITEM

In [28]:
cart_item_df.replace({pd.NaT: None}, inplace=True)
cart_item_dict = cart_item_df.to_dict(orient='records')

for ci in cart_item_dict:
    cart_item_rel = Relationship(order_details_dict_rel[ci["session_id"]], "cart_item", product_dict_rel[ci["product_id"]],cart_item_id=ci["cart_item_id"], session_id=ci["session_id"], product_id=ci["product_id"], quantity=ci["quantity"], created_at=ci["created_at"], modified_at=ci["modified_at"])

    graph.create(cart_item_rel)

### ORDER ITEMS

In [29]:
order_items_df.replace({pd.NaT: None}, inplace=True)

order_items_dict = order_items_df.to_dict(orient='records')
for oi in order_items_dict:
    order_items_rel = Relationship(order_details_dict_rel[oi["order_details_id"]], "order_items", product_dict_rel[oi["product_id"]], order_items_id=oi["order_items_id"], order_details_id=oi["order_details_id"], product_id=oi["product_id"], created_at=oi["created_at"], modified_at=oi["modified_at"])

    graph.create(order_items_rel)